<a href="https://colab.research.google.com/github/ChenJiajunPhilip/Agentic-AI/blob/main/Openrouter_LLM_5_letter_animal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Others

In [2]:
from openai import OpenAI
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
questions_file_path = '/content/drive/MyDrive/ARIA/week 2/LLM_Questions.txt'
responses_file_path = '/content/drive/MyDrive/ARIA/week 2/LLM_Responses.txt'

In [ ]:
# Read questions from LLM_Questions.txt
questions = []
try:
    with open(questions_file_path, 'r') as f:
        questions = f.readlines() #read line by line
    print(f"Successfully loaded {len(questions)} questions from {questions_file_path}")
except FileNotFoundError:
    print(f"Error: The file '{questions_file_path}' was not found.")

Successfully loaded 3 questions from /content/drive/MyDrive/ARIA/week 2/LLM_Questions.txt


#Main
1. Prompt model to provide a 5 letter animal word, check that word is valid.
2. Obtain fitness value of the word.
3. Prompt model providing information (current suggested word with fitness value, and also the fitness function),
4. prompt model again to provide an alternative animal word that has better fitness value.
5. Repeat steps 2 and 3 for a desired number of iterations.

In [3]:
OPENROUTER_API_KEY = "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" # Obtain unqiue code from your own Open Router account

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=OPENROUTER_API_KEY,
)

In [4]:
#with open(responses_file_path, 'w') as response_file: # each loop, 'w' will clear out conetnt from text file
#    for i, question in enumerate(questions):
#        question = question.strip()
#        if not question:
#            continue

#        print(f"\nProcessing Question {i+1}: {question}")

# Create some function (check_response) that verifies the output is a single word string of 5 characters
def find_fitness(s):
  f = 0
  if isinstance(s, str):
    for i in s.lower():
      f += ord(i) - ord('a') + 1
    return f
  else:
    return "N/A"

def check_response(response):
  """Checks if the response is a single 5-letter word."""
  if isinstance(response, str):
    words = response.split()
    #global fitness_value
    fitness_value = 0
    if len(words) == 1 and len(words[0]) == 5 and words[0].isalpha():
      for i in words[0].lower():
        fitness_value += ord(i) - ord('a') + 1
      return fitness_value > 70
  return False

#response_content = completion.choices[0].message.content
response_content = 0

# Implement the check and retry logic
max_retries = 10  # Set a limit for retries
retries = 0
while not check_response(response_content) and retries < max_retries:
    fitness_value = find_fitness(response_content)
    if response_content != 0:
      print(f"Response '{response_content}' did not meet criteria. fitness_value: {fitness_value}. Retrying...")
    retries += 1
    try:
        completion = client.chat.completions.create(
            extra_headers={
              "HTTP-Referer": "https://colab.research.google.com",
              "X-Title": "My Colab OpenRouter Gemini Test",
            },
            model="nousresearch/deephermes-3-mistral-24b-preview:free",
            messages=[
              {
                "role": "user",
                "content": "provide a single 5-letter animal word, and nothing else." #if (retries == 0) else (
                #"Try using another single 5-letter word with higher values, given a=1, b=2, ... z=26, but make sure that the 5 letter word is still a valid animal word.") # Refined prompt
              }
            ],
            max_tokens=10, # Reduce max tokens as you only expect one word
            temperature=0.7,
            top_p=0.8,
        )
        response_content = completion.choices[0].message.content
    except Exception as e:
        print(f"An error occurred during retry: {e}")
        break # Exit retry loop on error

if check_response(response_content):
    print(f"Valid Response: {response_content}")
    # Proceed with writing to file or other logic
else:
    print(f"Failed to get a valid 5-letter word after {max_retries} retries. Final response: {response_content}. fitness_value: {fitness_value}")
    # Handle the failure case (e.g., log an error, use a default word)



# 1. Prompt model to provide a 5 letter animal word, check that word is valid.
# 2. Obtain fitness value of the word.
# 3. Prompt model providing information (current suggested word with fitness value, and also the fitness function),
# prompt model again to provide an alternative animal word that has better fitness value.
# Repeat steps 2 and 3 for a desired number of iterations.

An error occurred during retry: Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day', 'code': 429, 'metadata': {'headers': {'X-RateLimit-Limit': '50', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1747958400000'}, 'provider_name': None}}, 'user_id': 'user_2x6uT7hWZv1MjIgEIepjnNrdlpG'}
Failed to get a valid 5-letter word after 10 retries. Final response: 0. fitness_value: N/A
